In [1]:
import json, ast
import openai
from dotenv import load_dotenv  
import os

# 加载.env文件  
load_dotenv("en1106.env")  

os.environ["OPENAI_API_TYPE"] = os.environ["Azure_OPENAI_API_TYPE1"]
os.environ["OPENAI_API_BASE"] = os.environ["Azure_OPENAI_API_BASE1"]
os.environ["OPENAI_API_KEY"] =  os.environ["Azure_OPENAI_API_KEY1"]
os.environ["OPENAI_API_VERSION"] = os.environ["Azure_OPENAI_API_VERSION1"]
BASE_URL=os.environ["OPENAI_API_BASE"]
API_KEY=os.environ["OPENAI_API_KEY"]

CHAT_DEPLOYMENT_NAME=os.environ.get('AZURE_OPENAI_API_CHAT_DEPLOYMENT_NAME')
EMBEDDING_DEPLOYMENT_NAME=os.environ.get('AZURE_OPENAI_API_EMBEDDING_DEPLOYMENT_NAME')

openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
# LLM
import requests
from urllib.parse import quote  
  

messages=[]
system_message = {"role":"system","content":'''You're python developer.And always create python function to resolve question"
    '''}
max_times=5
current_times=1

def ask(prompt):
    messages.append({ "role": "user","content":prompt})
    result = run_conversation()
    print(result)

def run_code(code_in_string,function_name,return_local_variable_name):
    try:
        print(f'{function_name} running...')
        exec(code_in_string,globals(),locals())
        return locals()[return_local_variable_name]
    except Exception as e:
        return f'error: {str(e)}'
    
def run_conversation():
    global current_times,max_times
    if current_times> max_times:
        return "Error running exceed the max loop times!"
    # Step 1: send the conversation and available functions to the model
    print(f'======run_conversation======')
    tools = [
        {
            "type": "function",
            "function": {
                "name": "run_code",
                "description": "run python code",
                "parameters": {
                    "type": "object",
                    "properties": {
                      "code_in_string": {
                        "type": "string",
                        "description": "The python code function writen in string"
                      },
                      "function_name": {
                        "type": "string",
                        "description": "The python code function name"
                      },  
                      "return_local_variable_name":{
                        "type": "string",
                        "description": "The python code return local variable name"
                      }
                    },
                    "required": ["code_in_string","function_name","return_local_variable_name"],
                },
            },
        }
    ]
    print(messages)
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo-1106",
        messages = [system_message]+messages[-10:],
        temperature=0.5,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        stream=True
    )
    ret=''
    function_name=''
    funArg=''
    callId=None
    for chunk in response:
        #print(chunk)
        if chunk.choices:
            if 'content' in chunk.choices[0].delta:
                c=chunk.choices[0].delta.content
                ret+=c
                
            if "tool_calls" in chunk.choices[0].delta:
                if 'id' in chunk.choices[0].delta.tool_calls[0]:
                    callId=chunk.choices[0].delta.tool_calls[0].id
                if 'function' in chunk.choices[0].delta.tool_calls[0]:
                    if 'name' in chunk.choices[0].delta.tool_calls[0].function:
                        function_name=chunk.choices[0].delta.tool_calls[0].function.name
                    if 'arguments' in chunk.choices[0].delta.tool_calls[0].function:   
                        funArg+=chunk.choices[0].delta.tool_calls[0].function.arguments
                    
                
    if ret !='':
        #print(ret)
        messages.append({ "role": "assistant","content":ret})
        return ret

    print(callId)
    print(function_name)
    print(funArg)
    
    # Step 2: check if the model wanted to call a function
    if function_name !='':
        assistant_reply={
          "role": "assistant",
          "tool_calls": [
            {
              "id":callId,
              "type": "function",
              "function": {
                "name": function_name,
                "arguments":funArg
              }
            }
          ],
          "content":""
        }
       
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            run_code.__name__: run_code,
        }  # only one function in this example, but you can have multiple
        function_args = json.loads(funArg)
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        # Step 4: send the info for each function call and function response to the model
        messages.append(assistant_reply)  # extend conversation with assistant's reply
        messages.append(
                {
                    "tool_call_id": callId,
                    "role": "tool",
                    "name": function_name,
                    "content": str(function_response) if function_response else '',
                }
            )  # extend conversation with function response
        #step 5 verify result
        with open(f'{function_args["function_name"]}.py', "w") as file:
            file.write(function_args["code_in_string"])
        current_times+=1
        return run_conversation()
        



In [2]:
ask("Hello!")

======run_conversation======
[{'role': 'user', 'content': 'Hello!'}]
Hi there! How can I assist you today?


In [3]:
ask("计算前10位斐波那契数列的和")

======run_conversation======
[{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hi there! How can I assist you today?'}, {'role': 'user', 'content': '计算前10位斐波那契数列的和'}]
call_JkJX5dCAngk9ExNg8Eu9mXUI
run_code
{"code_in_string":"def fibonacci_sequence(n):\n    fib = [0, 1]\n    for i in range(2, n):\n        fib.append(fib[i-1] + fib[i-2])\n    return fib\n\nsequence = fibonacci_sequence(10)\nsum_sequence = sum(sequence)\nsum_sequence","function_name":"fibonacci_sequence_sum","return_local_variable_name":"sum_sequence"}
fibonacci_sequence_sum running...
======run_conversation======
[{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hi there! How can I assist you today?'}, {'role': 'user', 'content': '计算前10位斐波那契数列的和'}, {'role': 'assistant', 'tool_calls': [{'id': 'call_JkJX5dCAngk9ExNg8Eu9mXUI', 'type': 'function', 'function': {'name': 'run_code', 'arguments': '{"code_in_string":"def fibonacci_sequence(n):\\n    fib = [0, 1]\\n    for i in range